In [26]:
import os
import pandas as pd
import pickle

from dotenv import load_dotenv
from routingGA import RoutingGA
from GALib import GALib
from data.addresses import addresses
from distancesAPI import MapsAPI

%reload_ext autoreload
%autoreload 2

load_dotenv()

True

In [27]:
refetch_distances = False
calculate_time = False

In [28]:
mapsAPI = MapsAPI(os.getenv('API_KEY'))
address_chunks = MapsAPI.split_in_chunks(addresses, 25)
distances = mapsAPI.split_distance_request(address_chunks, calculateTime = calculate_time) if refetch_distances else MapsAPI.get_from_cache()
distances = MapsAPI.convert_to_symmetric(distances)

In [29]:
df = pd.DataFrame(distances)

In [49]:
with open('cache/distance_matrix', 'rb') as f:
    best_ever = pickle.load(f)

In [50]:
routingGA = RoutingGA(popSize=200, qtyLocations=len(distances) - 1, qtyRoutes=3,
                          maxGenerations=800, selectionK=5, mutationRate=0.05, optRate=0.02,distances=distances)
                           
lib = GALib(routingGA=routingGA,
            libPath=os.getenv('LIB_PATH'))

best = None
for i in range(0, 10):
    routes = lib.run()
    distance = routingGA.calculate_total_distance(routes)
    print(distance)
    if distance <= routingGA.calculate_total_distance(best):
        best = routes

best

9791
9998
7567
9233
9056
13543
13448
8054
10319
13985


[[0, 1, 25],
 [0, 26, 23, 3],
 [0, 5, 8, 19, 4, 15, 7],
 [0,
  12,
  29,
  11,
  13,
  24,
  30,
  2,
  18,
  9,
  32,
  6,
  14,
  28,
  27,
  22,
  20,
  21,
  16,
  17,
  31,
  10]]

In [44]:
routingGA.calculate_total_distance(best)

7368

In [ ]:
# route_addresses = [[addresses[y] for y in x] for x in best]
# results = mapsAPI.split_directions_request(route_addresses[0])
# mapsAPI.save_to_image(results, f"route_test")

In [43]:
route_addresses = [[addresses[y] for y in x] for x in best]
imageCount = 1
for route in route_addresses:
    directions = mapsAPI.directions(route[0], route[0], route[1: 25])
    mapsAPI.save_to_image(directions, f"route_{imageCount}")
    imageCount += 1